In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [2]:
conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/11 17:53:30 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp4s0)
23/12/11 17:53:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 17:53:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/11 17:53:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
    StructField("bungee_audit_status", StringType(), True)
])

sku_uuid_schema =  StructType([
    StructField("sku_uuid", StringType(), True),
])

In [4]:
mdw_data = [
    [ "1<>homedepot<>homedepot_a<>bcg<>bcg","1<>homedepot<>homedepot", "homedepot<>homedepot","a<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "2<>homedepot<>homedepot_b<>bcg<>bcg","2<>homedepot<>homedepot", "homedepot<>homedepot","b<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "3<>homedepot<>homedepot_c<>bcg<>bcg","3<>homedepot<>homedepot", "homedepot<>homedepot","c<>bcg<>bcg","bcg<>bcg","3P","un_audited"],
    [ "4<>petcarerx<>petcarerx_d<>chewy<>chewy","4<>petcarerx<>petcarerx","petcarerx<>petcarerx","d<>chewy<>chewy", "chewy<>chewy","3P","pruned"],
    
    [ "7<>homedepot<>homedepot_e<>bcg<>bcg","7<>homedepot<>homedepot", "homedepot<>homedepot","e<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "8<>homedepot<>homedepot_f<>bcg<>bcg","8<>homedepot<>homedepot", "homedepot<>homedepot","f<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "9<>homedepot<>homedepot_g<>bcg<>bcg","9<>homedepot<>homedepot", "homedepot<>homedepot","g<>bcg<>bcg","bcg<>bcg","3P","un_audited"],
    [ "10<>petcarerx<>petcarerx_h<>chewy<>chewy","10<>petcarerx<>petcarerx","petcarerx<>petcarerx","h<>chewy<>chewy", "chewy<>chewy","3P","pruned"],
    
    [ "5<>homedepot<>homedepot_i<>bcg<>bcg","5<>homedepot<>homedepot","homedepot<>homedepot","i<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "5<>homedepot<>homedepot_j<>bcg<>bcg","5<>homedepot<>homedepot","homedepot<>homedepot","j<>bcg<>bcg", "bcg<>bcg","3P","similar"],

    [ "11<>homedepot<>homedepot_k<>bcg<>bcg","11<>homedepot<>homedepot", "homedepot<>homedepot","k<>bcg<>bcg","bcg<>bcg","1P","un_audited"],
    [ "12<>homedepot<>homedepot_k<>bcg<>bcg","12<>homedepot<>homedepot", "homedepot<>homedepot","k<>bcg<>bcg","bcg<>bcg","3P","pruned"],
]

In [5]:
deactivated_products = [
    ["a<>bcg<>bcg"],
    ["b<>bcg<>bcg"],
    ["c<>bcg<>bcg"],
    ["d<>chewy<>chewy"],
    ["5<>homedepot<>homedepot"],
    ["5<>homedepot<>homedepot"],
    ["k<>bcg<>bcg"],
    ["k<>bcg<>bcg"],
]

In [6]:
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)
mdw.show()
deactivated_products_df = spark.createDataFrame(deactivated_products, schema=sku_uuid_schema)
deactivated_products_df.show()

+--------------------+--------------------+--------------------+---------------+-----------------+-----------+-------------------+
|             pair_id|       base_sku_uuid|   base_source_store|  comp_sku_uuid|comp_source_store|seller_type|bungee_audit_status|
+--------------------+--------------------+--------------------+---------------+-----------------+-----------+-------------------+
|1<>homedepot<>hom...|1<>homedepot<>hom...|homedepot<>homedepot|    a<>bcg<>bcg|         bcg<>bcg|         1P|              exact|
|2<>homedepot<>hom...|2<>homedepot<>hom...|homedepot<>homedepot|    b<>bcg<>bcg|         bcg<>bcg|         1P|            similar|
|3<>homedepot<>hom...|3<>homedepot<>hom...|homedepot<>homedepot|    c<>bcg<>bcg|         bcg<>bcg|         3P|         un_audited|
|4<>petcarerx<>pet...|4<>petcarerx<>pet...|petcarerx<>petcarerx|d<>chewy<>chewy|     chewy<>chewy|         3P|             pruned|
|7<>homedepot<>hom...|7<>homedepot<>hom...|homedepot<>homedepot|    e<>bcg<>bcg|   

In [7]:
from deactivator import Match_Deactivator

match_deactivator = Match_Deactivator(mdw, deactivated_products_df)
deactivated_matches = match_deactivator.fetch_deactivate_matches()

Combining dfs


In [8]:
deactivated_matches.show()

+--------------------+--------------------+--------------------+---------------+-----------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+
|       base_sku_uuid|             pair_id|   base_source_store|  comp_sku_uuid|comp_source_store|seller_type|bungee_audit_status|           misc_info|      bungee_auditor|        updated_date|          updated_by|
+--------------------+--------------------+--------------------+---------------+-----------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+
|5<>homedepot<>hom...|5<>homedepot<>hom...|homedepot<>homedepot|    i<>bcg<>bcg|         bcg<>bcg|         1P|        deactivated|deactivated_base_...|match_management_...|match_management_...|2023-12-11 17:53:...|
|5<>homedepot<>hom...|5<>homedepot<>hom...|homedepot<>homedepot|    j<>bcg<>bcg|         bcg<>bcg|         3P|        deactivated|deactivate